In [1]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.2 MB/s eta 0:00:00


In [3]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib

df = pd.read_csv('dataset_sdn.csv', encoding='utf-8')

sql_injection_model_path = '/content/sql_injection_model.joblib'
sql_injection_model_pipeline = joblib.load(sql_injection_model_path)

ddos_attack_model_path = '/content/dtc_pipeline.joblib'
ddos_attack_model_pipeline = joblib.load(ddos_attack_model_path)

ndr_attack_model_path = '/content/etc_pipeline.joblib'
ndr_attack_model_pipeline = joblib.load(ndr_attack_model_path)
class_dict = {0:'Safe',1:'Suspicious'}

# Function to read YAML file as a dictionary
def data_processing(data):
    data['Time'] = pd.to_datetime(data['Time'], unit='s', origin='unix')
    data['Hour'] = data['Time'].dt.hour
    data['Minute'] = data['Time'].dt.minute
    data['Second'] = data['Time'].dt.second
    data['DayOfWeek'] = data['Time'].dt.dayofweek
    data['Day'] = data['Time'].dt.day

    # Convert Source and Destination IP addresses to numerical values using hashing
    data['Source_IP_Hash'] = data['Source'].apply(lambda x: hash(x) % (2**16))
    data['Destination_IP_Hash'] = data['Destination'].apply(lambda x: hash(x) % (2**16))

    # If Source Port or Destination Port have NaN values, fill them with a placeholder (e.g., 0 or -1)
    data['Source Port'] = data['Source Port'].fillna(-1).astype(int)
    data['Destination Port'] = data['Destination Port'].fillna(-1).astype(int)

    # Calculate traffic volume: count of packets per second
    data['Packets_per_Second'] = data.groupby('Second')['Length'].transform('count')

    # Calculate rolling features (e.g., rolling mean and std for Length over the past 5 packets)
    data['Rolling_Mean_Length'] = data['Length'].rolling(window=5).mean().fillna(data['Length'])
    data['Rolling_Std_Length'] = data['Length'].rolling(window=5).std().fillna(0)

    # Dropping original 'Time', 'Source', and 'Destination' columns as they've been transformed
    data = data.drop(['Time', 'Source', 'Destination'], axis=1)
    return data

st.title("Network attack detection system")
user_labels = ["sql","ddos","nad"]
user_choice = st.sidebar.selectbox(label = "Select model", options = user_labels, index=0)

if user_choice == user_labels[1]:
    st.image("/content/attack img.png")
    # Dropdowns for categorical inputs
    protocol = st.selectbox("Protocol", df['Protocol'].unique().tolist())
    src_ip = st.selectbox("Source IP", df['src'].unique().tolist())
    dst_ip = st.selectbox("Destination IP", df['dst'].unique().tolist())

    # Numeric inputs
    dt = st.number_input("DT")
    switch = st.number_input("Switch")
    packet_count = st.number_input("Packet Count")
    byte_count = st.number_input("Byte Count")
    duration = st.number_input("Duration")
    duration_ns = st.number_input("Duration (Nanoseconds)")
    flows = st.number_input("Flows")
    packet_ins = st.number_input("Packet Ins")
    packets_per_flow = st.number_input("Packets per Flow")
    pair_flow = st.number_input("Pair Flow")
    port_number = st.number_input("Port Number")
    tx_bytes = st.number_input("TX Bytes")
    rx_bytes = st.number_input("RX Bytes")
    tx_kbps = st.number_input("TX Kbps")
    rx_kbps = st.number_input("RX Kbps")
    total_kbps = st.number_input("Total Kbps")

    # Button to predict
    if st.button("Predict"):
        # Create a dictionary of inputs to pass to the prediction function
        input_data = pd.DataFrame({
            "dt": [dt],
            "switch": [switch],
            "src": [src_ip],
            "dst": [dst_ip],
            "pktcount": [packet_count],
            "bytecount": [byte_count],
            "dur": [duration],
            "dur_nsec": [duration_ns],
            "flows": [flows],
            "packetins": [packet_ins],
            "pktperflow": [packets_per_flow],
            "Pairflow": [pair_flow],
            "Protocol": [protocol],
            "port_no": [port_number],
            "tx_bytes": [tx_bytes],
            "rx_bytes": [rx_bytes],
            "tx_kbps": [tx_kbps],
            "rx_kbps": [rx_kbps],
            "tot_kbps": [total_kbps]
        })

        prediction = ddos_attack_model_pipeline.predict(input_data)[0]
        result = "Attack" if prediction == 1 else "No Attack"

        # Display the result
        st.write("Prediction:", result)

elif user_choice == user_labels[-1]:
    st.image("/content/attack img.png")
    sample_df = pd.read_csv("threat_data.csv")
    time = st.number_input("Time", value=sample_df['Time'].min(), step=0.1, min_value=sample_df['Time'].min(), max_value=sample_df['Time'].max())
    source = st.text_input("Source", value="192.232.16.204")
    destination = st.text_input("Destination", value="192.167.5.35")
    protocol = st.selectbox("Protocol", sample_df['Protocol'].unique())
    length = st.number_input("Length", value=float(sample_df['Length'].min()), step=0.1, min_value=float(sample_df['Length'].min()), max_value=float(sample_df['Length'].max()))
    source_port = st.number_input("Source Port", value=sample_df['Source Port'].min(), step=1.0, min_value=sample_df['Source Port'].min(), max_value=sample_df['Source Port'].max())
    destination_port = st.number_input("Destination Port", value=sample_df['Destination Port'].min(), step=1.0, min_value=sample_df['Destination Port'].min(), max_value=sample_df['Destination Port'].max())

    if st.button('Predict'):
        # Create a DataFrame from the input
        data = pd.DataFrame([{
            'Time': time,
            'Source': source,
            'Destination': destination,
            'Protocol': protocol,
            'Length': length,
            'Source Port': source_port,
            'Destination Port': destination_port,
        }])

        data = data_processing(data)

        pred_class = ndr_attack_model_pipeline.predict(data)
        st.markdown(f"Predicted class: **{pred_class[0]} ({class_dict[pred_class[0]]})**")
else:
    st.image("/content/attack img.png")
    text = st.text_input("Enter query")
    if st.button("Predict"):
        prediction = sql_injection_model_pipeline.predict([text])[0]
        st.write("Prediction:", prediction)

Overwriting app.py


# using pyngrok

In [4]:
!pip install pyngrok

In [5]:
from pyngrok import ngrok

ngrok_key = "2rbVgmHjKswONL4hUu2RzqtRwZ6_3rYhfeF8abrbYW7AGb3t6"
port = 8501

ngrok.set_auth_token(ngrok_key)
ngrok.connect(port).public_url

'https://b0a0-34-138-122-180.ngrok-free.app'

In [6]:
!rm -rf logs.txt && streamlit run app.py &>/content/logs.txt